#### Import nécessaire

In [1]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
from datetime import datetime
import sqlite3

#### Déclaraton des variables 

In [2]:
load_dotenv()
api_key = os.getenv("API_KEY")
url = "https://api.openweathermap.org/data/2.5/weather"
villes = ["Paris", "Marseille", "Lyon", "Toulouse", "Nice", "Nantes", "Montpellier", "Strasbourg", "Bordeaux", "Lille", "Rennes", "Reims", "Le Havre", "Saint-Étienne", "Toulon", "Grenoble", "Dijon", "Angers", "Nîmes", "Villeurbanne"]
data = []


In [3]:
print(api_key)

cdc6aff3f317ac1b879591530ef4c4ec


#### ***Avec Postman*** pour une meilleur visibilité ->  afficher un exemple de réponse en .json() pour voir comment est structuré le dictionnaire


In [4]:
r = requests.get(f"http://api.openweathermap.org/data/2.5/weather?q=Paris&units=metric&appid={api_key}")
r.json()

{'coord': {'lon': 2.3488, 'lat': 48.8534},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 10.84,
  'feels_like': 9.36,
  'temp_min': 9.32,
  'temp_max': 11.32,
  'pressure': 1024,
  'humidity': 53},
 'visibility': 10000,
 'wind': {'speed': 6.17, 'deg': 60},
 'clouds': {'all': 0},
 'dt': 1680545828,
 'sys': {'type': 2,
  'id': 2041230,
  'country': 'FR',
  'sunrise': 1680499491,
  'sunset': 1680546168},
 'timezone': 7200,
 'id': 2988507,
 'name': 'Paris',
 'cod': 200}

#### On voit que une grande partie des informations dont nous avons besoin sont regroupés dans  "main", "wind" et "sys" avec comme valeur (Pour PARIS) : 
                        main : 
        "temp": 10.3, -> (Température actuelle)
        "feels_like": 8.8, ->  (Température ressentie)
        "temp_min": 9.5, -> (Température minimale)
        "temp_max": 11.25, -> (empérature maximale)
        "pressure": 1026, -> (Pression atmosphérique)
        "humidity": 54 -> (Humidité)

                        wind : 
        "speed": 7.72, -> (Vitesse du vent)
        "deg": 40 -> (Direction du vent)
        
                        sys : 
        "sunrise": 1680499491, -> (Lever du soleil)
        "sunset": 1680546168 -> (Coucher du soleil)

#### Creation du DataFrame regroupant les informations de météo par ville + conversion de certains données

In [5]:
for ville in villes:
    params = {
        "q": ville + ",FR",
        "appid": api_key,
        "units": "metric", 
        "lang": "fr"
    }
# utilisation de "units": "metric" ici pour convertir directement les unités de mesure (km/h et C°) pour faciliter la compréhension.

    reponse = requests.get(url, params=params)
    
    if reponse.status_code == 200:
        rep_json = reponse.json()
        
        meteo_france= {
            "Ville": ville,
            "Température actuelle C°": rep_json["main"]["temp"],
            "Température ressentie C°": rep_json["main"]["feels_like"],
            "Température minimale C°": rep_json["main"]["temp_min"],
            "Température maximale C°": rep_json["main"]["temp_max"],
            "Pression atmosphérique": rep_json["main"]["pressure"],
            "Humidité": rep_json["main"]["humidity"],
            "Vitesse du vent km/h": rep_json["wind"]["speed"],
            "Direction du vent en ° ": rep_json["wind"]["deg"],
            "Lever du soleil": datetime.fromtimestamp(rep_json["sys"]["sunrise"]).strftime('%d-%m-%Y %H:%M:%S'),
            "Coucher du soleil": datetime.fromtimestamp(rep_json["sys"]["sunset"]).strftime('%d-%m-%Y %H:%M:%S')
        }
# Utilisation de fromtimestamp() et strftime() ici pour prendre, dans un premier temps le Timestamp UNIX et renvoyer le "datetime" correspondant.
# Ensuite transormer le "datetime" dans le format souhaité pour une meilleur compréhension avec "strftime".
        data.append(meteo_france)
    else:
        print(reponse.status_code)

In [6]:
df_top20fr = pd.DataFrame(data)
# Pour définir Paris comme l'index1 (0 avant ce code)
df_top20fr = df_top20fr.reset_index()
df_top20fr.index += 1
df_top20fr = df_top20fr.drop(columns=['index'])

In [7]:
df_top20fr

,Ville,Température actuelle C°,Température ressentie C°,Température minimale C°,Température maximale C°,Pression atmosphérique,Humidité,Vitesse du vent km/h,Direction du vent en °,Lever du soleil,Coucher du soleil
1,Paris,10.84,9.36,9.32,11.32,1024,53,6.17,60,03-04-2023 07:24:51,03-04-2023 20:22:48
2,Marseille,11.70,10.39,10.10,13.25,1008,56,4.47,333,03-04-2023 07:16:58,03-04-2023 20:05:29
3,Lyon,5.42,2.01,5.03,7.04,1018,72,4.69,343,03-04-2023 07:18:42,03-04-2023 20:11:05
4,Toulouse,11.92,10.79,11.01,12.09,1017,62,5.14,290,03-04-2023 07:32:58,03-04-2023 20:21:56
5,Nice,6.35,6.35,3.75,8.34,1010,73,1.01,189,03-04-2023 07:09:27,03-04-2023 19:58:59
6,Nantes,9.83,7.74,8.72,9.96,1023,66,4.12,60,03-04-2023 07:42:06,03-04-2023 20:37:00
7,Montpellier,14.42,12.94,13.29,14.76,1011,39,8.75,10,03-04-2023 07:23:14,03-04-2023 20:12:11
8,Strasbourg,6.16,2.72,5.31,7.04,1025,52,5.14,50,03-04-2023 07:03:34,03-04-2023 20:00:56
9,Bordeaux,12.15,11.04,12.15,12.41,1019,62,7.20,30,03-04-2023 07:40:04,03-04-2023 20:31:01
10,Lille,8.55,5.72,6.50,9.41,1026,43,5.14,40,03-04-2023 07:20:16,03-04-2023 20:21:42


### Exporter le DF en CSV

In [8]:
df_top20fr.to_csv("meteo_france_top20.csv", index=False)


### Creation de Base de données SQL

In [9]:
connexion = sqlite3.connect("meteo_fr_top20.db")

connexion.execute("""
CREATE TABLE IF NOT EXISTS données_météo (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Ville TEXT,
    Température_actuelle REAL,
    Température_ressentie REAL,
    Température_minimale REAL,
    Température_maximale REAL,
    Pression_atmosphérique INTEGER,
    Humidité INTEGER,
    Vitesse_du_vent REAL,
    Direction_du_vent INTEGER,
    Lever_du_soleil TEXT,
    Coucher_du_soleil TEXT
);
""")

for index, row in df_top20fr.iterrows():
    connexion.execute(
        """
        INSERT INTO données_météo (Ville, Température_actuelle, Température_ressentie, Température_minimale, Température_maximale,
        Pression_atmosphérique, Humidité, Vitesse_du_vent, Direction_du_vent, Lever_du_soleil, Coucher_du_soleil)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
         """, 
    (row["Ville"], row["Température actuelle C°"], row["Température ressentie C°"], row["Température minimale C°"], row["Température maximale C°"], row["Pression atmosphérique"],
     row["Humidité"], row["Vitesse du vent km/h"], row["Direction du vent en ° "], row["Lever du soleil"], row["Coucher du soleil"]))

connexion.commit()
connexion.close()
